In [ ]:
!pip install transformers torch tqdm

import torch
from transformers import GPT2Tokenizer, GPT2Model
import numpy as np
import json
from tqdm import tqdm
from google.colab import files

In [2]:
def generate_node_dict():
    descriptions = {
        "__background__": {
            "description": "Background or undefined region",
            "placement": "Any location serving as context or environment"
        },
        "airplane": {
            "description": "Large metal transport vehicle with wings and engines",
            "placement": "In sky regions, on airport runways, or at terminal gates"
        },
        "animal": {
            "description": "Generic living creature",
            "placement": "In natural habitats, varying by species and environment"
        },
        "arm": {
            "description": "Human body limb",
            "placement": "Natural poses relative to human body, gesturing or interacting"
        },
        "bag": {
            "description": "Container for carrying items",
            "placement": "Held by handles, on floors, or hanging from hooks"
        },
        "banana": {
            "description": "Curved yellow fruit",
            "placement": "In fruit bowls, on counters, or grouped with other fruits"
        },
        "basket": {
            "description": "Woven container",
            "placement": "On floors, tables, or hanging from surfaces"
        },
        "beach": {
            "description": "Sandy shoreline",
            "placement": "Along water edges, with appropriate horizon line"
        },
        "bear": {
            "description": "Large furry mammal",
            "placement": "In wilderness settings, near trees or water sources"
        },
        "bed": {
            "description": "Furniture for sleeping",
            "placement": "Against walls in rooms, with appropriate bedding"
        },
        "bench": {
            "description": "Long seating furniture",
            "placement": "Along paths, in parks, or against walls"
        },
        "bike": {
            "description": "Two-wheeled pedal vehicle",
            "placement": "On paths, leaning against structures, or in racks"
        },
        "bird": {
            "description": "Flying creature",
            "placement": "On branches, in flight, or on ground"
        },
        "board": {
            "description": "Flat rigid surface",
            "placement": "Mounted on walls or supported on surfaces"
        },
        "boat": {
            "description": "Water vessel",
            "placement": "On water surfaces, at docks, or moored appropriately"
        },
        "book": {
            "description": "Bound paper object",
            "placement": "On shelves, tables, or in hands"
        },
        "boot": {
            "description": "Tall footwear",
            "placement": "On feet, floors, or in closets"
        },
        "bottle": {
            "description": "Container with neck",
            "placement": "Upright on surfaces, in holders"
        },
        "bowl": {
            "description": "Concave container",
            "placement": "On flat surfaces with stable base"
        },
        "box": {
            "description": "Rectangular container",
            "placement": "On floors, shelves, or stacked"
        },
        "boy": {
            "description": "Young male person",
            "placement": "Standing, sitting, or engaged in activities"
        },
        "branch": {
            "description": "Tree limb",
            "placement": "Extending from trees, natural angles"
        },
        "building": {
            "description": "Constructed structure",
            "placement": "On ground with appropriate perspective"
        },
        "bus": {
            "description": "Large passenger vehicle",
            "placement": "On roads, at stops, proper scale"
        },
        "cabinet": {
            "description": "Storage furniture",
            "placement": "Against walls, proper height and alignment"
        },
        "cap": {
            "description": "Head covering",
            "placement": "On heads, hooks, or surfaces"
        },
        "car": {
            "description": "Passenger vehicle",
            "placement": "On roads, in parking spaces, correct orientation"
        },
        "cat": {
            "description": "Feline pet",
            "placement": "On furniture, floors, or in typical poses"
        },
        "chair": {
            "description": "Single-person seat",
            "placement": "At tables, desks, or grouped appropriately"
        },
        "child": {
            "description": "Young person",
            "placement": "In age-appropriate contexts and activities"
        },
        "clock": {
            "description": "Time-telling device",
            "placement": "On walls, surfaces, or integrated in scenes"
        },
        "coat": {
            "description": "Outer garment",
            "placement": "On people, hooks, or hangers"
        },
        "counter": {
            "description": "Work surface",
            "placement": "Along walls, in kitchens or retail spaces"
        },
        "cow": {
            "description": "Large farm animal",
            "placement": "In fields, barns, or pastoral settings"
        },
        "cup": {
            "description": "Drinking vessel",
            "placement": "On flat surfaces, in hands, proper orientation"
        },
        "curtain": {
            "description": "Window covering",
            "placement": "Hanging from rods, framing windows"
        },
        "desk": {
            "description": "Work furniture",
            "placement": "Against walls, in rooms, with chairs"
        },
        "dog": {
            "description": "Canine pet",
            "placement": "On floors, furniture, or in active poses"
        },
        "door": {
            "description": "Entry portal",
            "placement": "In walls, proper scale and alignment"
        },
        "drawer": {
            "description": "Sliding storage",
            "placement": "Within furniture, proper height"
        },
        "ear": {
            "description": "Body part",
            "placement": "On heads, proper anatomical position"
        },
        "elephant": {
            "description": "Large mammal",
            "placement": "In open spaces, natural habitats"
        },
        "engine": {
            "description": "Mechanical device",
            "placement": "Within vehicles or machines"
        },
        "eye": {
            "description": "Visual organ",
            "placement": "On faces, proper anatomical position"
        },
        "face": {
            "description": "Front of head",
            "placement": "On people, proper orientation"
        },
        "fence": {
            "description": "Boundary structure",
            "placement": "Along property lines, consistent height"
        },
        "finger": {
            "description": "Hand digit",
            "placement": "On hands, natural gestures"
        },
        "flag": {
            "description": "Cloth symbol",
            "placement": "On poles, buildings, or moving in wind"
        },
        "flower": {
            "description": "Blooming plant",
            "placement": "In gardens, vases, or natural settings"
        },
        "food": {
            "description": "Edible items",
            "placement": "On plates, tables, or appropriate surfaces"
        },
        "fork": {
            "description": "Eating utensil",
            "placement": "On tables, in place settings"
        },
        "fruit": {
            "description": "Edible produce",
            "placement": "In bowls, on surfaces, or on plants"
        },
        "giraffe": {
            "description": "Tall spotted animal",
            "placement": "In savannas, near trees"
        },
        "girl": {
            "description": "Young female person",
            "placement": "In age-appropriate activities"
        },
        "glass": {
            "description": "Transparent container",
            "placement": "On tables, in hands, upright"
        },
        "glove": {
            "description": "Hand covering",
            "placement": "On hands, surfaces, or paired"
        },
        "guy": {
            "description": "Male person",
            "placement": "In various activities and poses"
        },
        "hair": {
            "description": "Head covering",
            "placement": "On heads, natural flow and style"
        },
        "hand": {
            "description": "Body extremity",
            "placement": "At end of arms, natural poses"
        },
        "handle": {
            "description": "Gripping part",
            "placement": "On objects, proper attachment"
        },
        "hat": {
            "description": "Head covering",
            "placement": "On heads, hooks, or surfaces"
        },
        "head": {
            "description": "Upper body part",
            "placement": "On bodies, natural orientation"
        },
        "helmet": {
            "description": "Protective headgear",
            "placement": "On heads, proper fit"
        },
        "hill": {
            "description": "Elevated ground",
            "placement": "In landscapes, natural slopes"
        },
        "horse": {
            "description": "Large equine",
            "placement": "In fields, stables, or trails"
        },
        "house": {
            "description": "Dwelling structure",
            "placement": "On ground, proper scale"
        },
        "jacket": {
            "description": "Body covering",
            "placement": "On people, hooks, or furniture"
        },
        "jean": {
            "description": "Denim pants",
            "placement": "On legs, hangers, or folded"
        },
        "kid": {
            "description": "Young person",
            "placement": "In appropriate activities"
        },
        "kite": {
            "description": "Flying toy",
            "placement": "In sky, proper string angle"
        },
        "lady": {
            "description": "Female person",
            "placement": "In various activities and poses"
        },
        "lamp": {
            "description": "Light fixture",
            "placement": "On surfaces, proper illumination"
        },
        "laptop": {
            "description": "Portable computer",
            "placement": "On flat surfaces, proper angle"
        },
        "leaf": {
            "description": "Plant part",
            "placement": "On plants, ground, or floating"
        },
        "leg": {
            "description": "Body limb",
            "placement": "Below torso, natural stance"
        },
        "letter": {
            "description": "Written message",
            "placement": "In hands, on surfaces"
        },
        "light": {
            "description": "Illumination source",
            "placement": "On ceilings, walls, or fixtures"
        },
        "logo": {
            "description": "Brand symbol",
            "placement": "On products, signs, or surfaces"
        },
        "man": {
            "description": "Adult male",
            "placement": "In various poses and activities"
        },
        "men": {
            "description": "Multiple males",
            "placement": "Grouped in activities"
        },
        "motorcycle": {
            "description": "Two-wheeled vehicle",
            "placement": "On roads, proper balance"
        },
        "mountain": {
            "description": "Large landform",
            "placement": "In landscapes, proper scale"
        },
        "mouth": {
            "description": "Facial feature",
            "placement": "On faces, proper position"
        },
        "neck": {
            "description": "Body part",
            "placement": "Between head and shoulders"
        },
        "nose": {
            "description": "Facial feature",
            "placement": "On faces, central position"
        },
        "number": {
            "description": "Numerical symbol",
            "placement": "On surfaces, signs"
        },
        "orange": {
            "description": "Citrus fruit",
            "placement": "In bowls, on surfaces"
        },
        "pant": {
            "description": "Leg clothing",
            "placement": "On legs, hangers, or folded"
        },
        "paper": {
            "description": "Flat material",
            "placement": "On surfaces, in hands"
        },
        "paw": {
            "description": "Animal foot",
            "placement": "On animal legs, natural pose"
        },
        "people": {
            "description": "Multiple persons",
            "placement": "In groups, various activities"
        },
        "person": {
            "description": "Human being",
            "placement": "In various poses and settings"
        },
        "phone": {
            "description": "Communication device",
            "placement": "In hands, on surfaces"
        },
        "pillow": {
            "description": "Soft cushion",
            "placement": "On furniture, proper arrangement"
        },
        "pizza": {
            "description": "Round food",
            "placement": "On plates, in boxes"
        },
        "plane": {
            "description": "Aircraft",
            "placement": "In sky, on runways"
        },
        "plant": {
            "description": "Growing organism",
            "placement": "In soil, pots, or ground"
        },
        "plate": {
            "description": "Flat dish",
            "placement": "On tables, proper setting"
        },
        "player": {
            "description": "Game participant",
            "placement": "In active poses"
        },
        "pole": {
            "description": "Vertical support",
            "placement": "In ground, proper angle"
        },
        "post": {
            "description": "Vertical structure",
            "placement": "In ground, supporting items"
        },
        "pot": {
            "description": "Cooking vessel",
            "placement": "On stoves, surfaces"
        },
        "racket": {
            "description": "Sports equipment",
            "placement": "In hands, proper grip"
        },
        "railing": {
            "description": "Safety barrier",
            "placement": "Along edges, proper height"
        },
        "rock": {
            "description": "Natural stone",
            "placement": "On ground, natural grouping"
        },
        "roof": {
            "description": "Building top",
            "placement": "On structures, proper angle"
        },
        "room": {
            "description": "Indoor space",
            "placement": "Within buildings, proper scale"
        },
        "screen": {
            "description": "Display surface",
            "placement": "On devices, walls"
        },
        "seat": {
            "description": "Sitting surface",
            "placement": "In vehicles, proper height"
        },
        "sheep": {
            "description": "Woolly animal",
            "placement": "In fields, groups"
        },
        "shelf": {
            "description": "Storage surface",
            "placement": "On walls, proper level"
        },
        "shirt": {
            "description": "Upper garment",
            "placement": "On torsos, hangers"
        },
        "shoe": {
            "description": "Footwear",
            "placement": "On feet, floors, paired"
        },
        "short": {
            "description": "Brief pants",
            "placement": "On legs, folded"
        },
        "sidewalk": {
            "description": "Paved path",
            "placement": "Along streets, proper width"
        },
        "sign": {
            "description": "Information display",
            "placement": "On posts, walls"
        },
        "sink": {
            "description": "Washing basin",
            "placement": "In counters, proper height"
        },
        "skateboard": {
            "description": "Rolling board",
            "placement": "Under feet, on ground"
        },
        "ski": {
            "description": "Snow equipment",
            "placement": "On snow, proper angle"
        },
        "skier": {
            "description": "Snow athlete",
            "placement": "On skis, in motion"
        },
        "sneaker": {
            "description": "Athletic shoe",
            "placement": "On feet, floors"
        },
        "snow": {
            "description": "Frozen precipitation",
            "placement": "On ground, surfaces"
        },
        "sock": {
            "description": "Foot covering",
            "placement": "On feet, paired"
        },
        "stand": {
            "description": "Support structure",
            "placement": "On ground, holding items"
        },
        "street": {
            "description": "Paved road",
            "placement": "Between buildings, proper width"
        },
        "surfboard": {
            "description": "Water sport equipment",
            "placement": "On water, carried"
        },
        "table": {
            "description": "Flat furniture",
            "placement": "On floor, proper height"
        },
        "tail": {
            "description": "Animal appendage",
            "placement": "On animals, natural pose"
        },
        "tie": {
            "description": "Neck accessory",
            "placement": "Around necks, proper drape"
        },
        "tile": {
            "description": "Floor covering",
            "placement": "On surfaces, pattern"
        },
        "tire": {
            "description": "Wheel part",
            "placement": "On vehicles, proper rotation"
        },
        "toilet": {
            "description": "Bathroom fixture",
            "placement": "Against walls, proper height"
        },
        "towel": {
            "description": "Drying cloth",
            "placement": "On racks, folded"
        },
        "tower": {
            "description": "Tall structure",
            "placement": "On ground, vertical"
        },
        "track": {
            "description": "Path surface",
            "placement": "On ground, proper curve"
        },
        "train": {
            "description": "Rail vehicle",
            "placement": "On tracks, proper scale"
        },
        "tree": {
            "description": "Tall plant",
            "placement": "In ground, natural growth"
        },
        "truck": {
            "description": "Large vehicle",
            "placement": "On roads, proper scale"
        },
        "trunk": {
            "description": "Storage box",
            "placement": "In cars, proper fit"
        },
        "umbrella": {
            "description": "Rain shield",
            "placement": "Over heads, proper angle"
        },
        "vase": {
            "description": "Flower container",
            "placement": "On surfaces, proper stability"
        },
        "vegetable": {
            "description": "Edible plant",
            "placement": "On plates, in gardens"
        },
        "vehicle": {
            "description": "Transportation device",
            "placement": "On roads, proper orientation"
        },
        "wave": {
            "description": "Water motion",
            "placement": "On water surface, natural flow"
        },
        "wheel": {
            "description": "Circular part",
            "placement": "On vehicles, proper rotation"
        },
        "window": {
            "description": "Glass opening",
            "placement": "In walls, proper frame"
        },
        "windshield": {
            "description": "Vehicle glass",
            "placement": "On vehicles, proper angle"
        },
        "wing": {
            "description": "Flying appendage",
            "placement": "On aircraft, birds"
        },
        "wire": {
            "description": "Metal cord",
            "placement": "Between points, proper tension"
        },
        "woman": {
            "description": "Adult female",
            "placement": "In various poses and activities"
        },
        "zebra": {
            "description": "Striped equine",
            "placement": "In grasslands, groups"
        },
        "apple": {
            "description": "Round fruit",
            "placement": "In bowls, on surfaces"
        },
        "bicycle": {
            "description": "Pedal vehicle",
            "placement": "On paths, properly balanced"
        },
        "broccoli": {
            "description": "Green vegetable",
            "placement": "On plates, in cooking"
        },
        "cake": {
            "description": "Sweet dessert",
            "placement": "On plates, stands"
        },
        "cell phone": {
            "description": "Mobile device",
            "placement": "In hands, on surfaces"
        },
        "donut": {
            "description": "Ring pastry",
            "placement": "On plates, in boxes"
        },
        "fire hydrant": {
            "description": "Water access",
            "placement": "Along streets, proper access"
        },
        "keyboard": {
            "description": "Input device",
            "placement": "On desks, proper angle"
        },
        "knife": {
            "description": "Cutting tool",
            "placement": "On tables, in blocks"
        },
        "mouse": {
            "description": "Computer tool",
            "placement": "Beside keyboards, proper reach"
        },
        "potted plant": {
            "description": "Container flora",
            "placement": "On surfaces, proper light"
        },
        "remote": {
            "description": "Control device",
            "placement": "On furniture, in reach"
        },
        "sandwich": {
            "description": "Stacked food",
            "placement": "On plates, proper assembly"
        },
        "scissors": {
            "description": "Cutting tool",
            "placement": "On surfaces, proper handling"
        },
        "spoon": {
            "description": "Eating tool",
            "placement": "On tables, in settings"
        },
        "suitcase": {
            "description": "Travel container",
            "placement": "On ground, proper handle"
        },
        "toothbrush": {
            "description": "Cleaning tool",
            "placement": "In holders, proper angle"
        },
        "wine glass": {
            "description": "Stemmed vessel",
            "placement": "On tables, proper stability"
        }
    }
    return descriptions

In [ ]:
def generate_and_download_embeddings():
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    model = GPT2Model.from_pretrained('gpt2')
    tokenizer.pad_token = tokenizer.eos_token
    model.eval()

    # Generate node embeddings
    node_dict = generate_node_dict()
    node_embeddings = {}
    print("Generating node embeddings...")
    for node, node_data in tqdm(node_dict.items()):
        category_inputs = tokenizer(node, padding='max_length', max_length=128,
                                    truncation=True, return_tensors='pt')
        description_inputs = tokenizer(node_data['description'], padding='max_length', max_length=128,
                                       truncation=True, return_tensors='pt')
        cat_desc_inputs = tokenizer(f"{node}: {node_data['description']}", padding='max_length', max_length=128,
                                    truncation=True, return_tensors='pt')
        placement_inputs = tokenizer(node_data['placement'], padding='max_length', max_length=128,
                                     truncation=True, return_tensors='pt')
        cat_place_inputs = tokenizer(f"{node}. placement: {node_data['placement']}", padding='max_length', max_length=128,
                                     truncation=True, return_tensors='pt')
        cat_desc_place_inputs = tokenizer(f"{node}. description: {node_data['description']}. placement: {node_data['placement']}",
                                          padding='max_length', max_length=128, truncation=True, return_tensors='pt')

        with torch.no_grad():
            category_outputs = model(**category_inputs)
            description_outputs = model(**description_inputs)
            cat_desc_outputs = model(**cat_desc_inputs)
            placement_outputs = model(**placement_inputs)
            cat_place_outputs = model(**cat_place_inputs)
            cat_desc_place_outputs = model(**cat_desc_place_inputs)

            category_embedding = category_outputs.last_hidden_state.mean(dim=1).numpy().squeeze()
            description_embedding = description_outputs.last_hidden_state.mean(dim=1).numpy().squeeze()
            cat_desc_embedding = cat_desc_outputs.last_hidden_state.mean(dim=1).numpy().squeeze()
            placement_embedding = placement_outputs.last_hidden_state.mean(dim=1).numpy().squeeze()
            cat_place_embedding = cat_place_outputs.last_hidden_state.mean(dim=1).numpy().squeeze()
            cat_desc_place_embedding = cat_desc_place_outputs.last_hidden_state.mean(dim=1).numpy().squeeze()

            node_embeddings[node] = {
                'category_embedding': category_embedding,
                'description_embedding': description_embedding,
                'cat_desc_embedding': cat_desc_embedding,
                'placement_embedding': placement_embedding,
                'cat_place_embedding': cat_place_embedding,
                'cat_desc_place_embedding': cat_desc_place_embedding
            }

    # Save files
    np.save('node_embeddings.npy', node_embeddings)

    # Download files
    files.download('node_embeddings.npy')

generate_and_download_embeddings()

In [4]:
embeddings = np.load('node_embeddings.npy', allow_pickle=True).item()

In [ ]:
type(embeddings)

In [ ]:
embeddings.keys()

In [ ]:
type(embeddings["airplane"])

In [ ]:
embeddings["airplane"].keys()

In [10]:
def generate_edge_descriptions():
    descriptions = {
        "__background__": "No specific spatial or semantic relationship between objects.",
        "above": "Object positioned at higher elevation than reference object, with clear vertical separation.",
        "across": "Object located on opposite side from reference point, spanning horizontal distance.",
        "against": "Object in direct physical contact with reference object, often with supporting pressure.",
        "along": "Object following parallel path or linear alignment with reference object.",
        "and": "Objects coexisting or grouped together with equal importance.",
        "at": "Object located at specific position relative to reference point.",
        "attached to": "Object physically connected or fixed to reference object.",
        "behind": "Object positioned at rear of reference object, partially or fully occluded.",
        "belonging to": "Object owned by or associated with reference object or person.",
        "between": "Object positioned in space separating two reference points.",
        "carrying": "Object being transported or held by animate reference object.",
        "covered in": "Object completely overlaid or enveloped by reference material.",
        "covering": "Object providing overlay or protection to reference object.",
        "eating": "Object being consumed by animate reference object.",
        "flying in": "Object moving through reference space via aerial motion.",
        "for": "Object serving purpose or function for reference object.",
        "from": "Object originating or extending from reference point.",
        "growing on": "Object naturally developing or sprouting from reference surface.",
        "hanging from": "Object suspended downward from reference point.",
        "has": "Object possessed by or integral to reference object.",
        "holding": "Object being grasped or supported by reference object.",
        "in": "Object contained within boundaries of reference object.",
        "in front of": "Object positioned before reference object, fully visible.",
        "laying on": "Object positioned horizontally upon reference surface.",
        "looking at": "Object being viewed by animate reference object.",
        "lying on": "Object positioned horizontally upon reference surface.",
        "made of": "Object composed of reference material.",
        "mounted on": "Object firmly affixed to reference surface.",
        "near": "Object positioned in close proximity to reference object.",
        "of": "Object having inherent relationship to reference object.",
        "on": "Object supported by and in contact with reference surface.",
        "on back of": "Object positioned on posterior side of reference object.",
        "over": "Object positioned above reference object with possible coverage.",
        "painted on": "Object depicted via paint on reference surface.",
        "parked on": "Vehicle stationed on reference surface.",
        "part of": "Object forming component of larger reference object.",
        "playing": "Object engaged in recreational activity with reference object.",
        "riding": "Object being transported atop moving reference object.",
        "says": "Object displaying or communicating reference text.",
        "sitting on": "Object positioned in seated pose on reference surface.",
        "standing on": "Object positioned upright on reference surface.",
        "to": "Object directed toward or reaching reference point.",
        "under": "Object positioned below reference object.",
        "using": "Object being utilized by reference object for purpose.",
        "walking in": "Object moving by foot within reference space.",
        "walking on": "Object moving by foot upon reference surface.",
        "watching": "Object observing or monitoring reference object.",
        "wearing": "Object adorned or clothed by reference object.",
        "wears": "Object being worn by reference object.",
        "with": "Object accompanying or paired with reference object."
    }
    return descriptions

In [ ]:
def generate_and_download_edge_embeddings():
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    model = GPT2Model.from_pretrained('gpt2')
    tokenizer.pad_token = tokenizer.eos_token
    model.eval()

    edge_descriptions = generate_edge_descriptions()
    edge_embeddings = {}
    print("Generating edge embeddings...")
    for edge, desc in tqdm(edge_descriptions.items()):
        inputs = tokenizer(desc, padding='max_length', max_length=128,
                           truncation=True, return_tensors='pt')
        with torch.no_grad():
            outputs = model(**inputs)
            embedding = outputs.last_hidden_state.mean(dim=1)
            edge_embeddings[edge] = embedding.numpy().squeeze()

    np.save('edge_embeddings.npy', edge_embeddings)
    with open('edge_descriptions.json', 'w') as f:
        json.dump(edge_descriptions, f, indent=2)

    files.download('edge_embeddings.npy')

generate_and_download_edge_embeddings()

In [12]:
embeddings = np.load('edge_embeddings.npy', allow_pickle=True).item()

In [ ]:
type(embeddings)

In [ ]:
embeddings.keys()

In [ ]:
type(embeddings['above'])

In [ ]:
embeddings['above'].shape

In [ ]:
embeddings['above'][:20]